In [55]:
import pandas as pd # Beautiful tool for data wrangling! e.g. '!pip install pandas' from a Notebook
# See https://mariadb.com/blog/how-connect-python-programs-mariadb e.g. '!pip install mysql' from Notebook
import mysql.connector as mariadb 
import re
pd.set_option("display.max_rows",35) # Useful when having large Pandas DataFrames like we do here

In [2]:
conn = mariadb.connect(user='mos', password='', database='monuments_db',buffered=True)
cursor = conn.cursor()

### Monuments_all

In [112]:
cursor.execute("SELECT * FROM monuments_all WHERE country='se-bbr'")
all_bbr = pd.io.sql.read_sql('select * from monuments_all WHERE country="se-bbr"', conn)
all_bbr.shape

(5686, 22)

### monuments_all_se-bbr_(sv)

In [3]:
table_name = "se_bbr" # I've renamed monuments_se-bbr_(se) to 'se_bbr' in local database, change to correct name
se_bbr = pd.io.sql.read_sql('select * from se_bbr', conn)
se_bbr.shape

(5686, 16)

In [115]:
se_bbr.keys()

Index(['bbr', 'namn', 'region-iso', 'funktion', 'byggar', 'arkitekt', 'plats',
       'kommun', 'lat', 'lon', 'bild', 'commonscat', 'source', 'changed',
       'monument_article', 'registrant_url'],
      dtype='object')

In [81]:
se_bbr.ix[10:20, ["namn","funktion"]]

,namn,funktion
10,"[[Huddungeby]], [[Anders Olsgården]] (Huddunge...",Bondgård (2 byggnader)
11,[[Harbonäs]] säteri (Harbonäs 2:1),Säteri (1 byggnad)
12,"[[Huddungeby]], [[Jan Ers-gården]] (Huddungeby...",Bondgård (4 byggnader)
13,[[Östervåla gästgivargård]] (Östervåla-Åby 8:6...,Gästgivargård (1 byggnad)
14,[[Biskops-Arnö]] (Biskops-Arnö 1:1),Biskopsgård (9 byggnader)
15,[[Skoklosters slott]] (Skokloster 2:4; f.d. Sk...,Slott (5 byggnader)
16,[[Flottsunds fyr]] (Kungshamn 1-5:1),Fyrplats (1 byggnad)
17,"[[Krusenbergs herrgård]] (Krusenberg 1:58, 1:1...","Herrgård,Säteri (5 byggnader)"
18,[[Mora stenar]] (Mora 1:1),Museum (1 byggnad)
19,[[Skottsila]] f.d. mönsterskrivarboställe (Sko...,Ämbets- och tjänstemannaboställe (4 byggnader)


In [107]:
first = se_bbr.groupby("kommun")[["funktion","namn"]].first()
first.head()

,funktion,namn
kommun,,
Ale,Repslageri (1 byggnad),"[[Repslagarmuseet]] (Starrkärr 15:4, Carlmarks..."
Alingsås,Rådhus (1 byggnad),[[Alingsås rådhus]] (Kristina 7; f.d. Kristina 3)
Alvesta,Järnvägsstation (1 byggnad),[[Alvesta stationshus]] (Alvesta 11:1)
Aneby,"Herrgård, säteri (1 byggnad)",[[Degla gård]] (Degla 1:1)
Arboga,Kungsgård (11 byggnader),[[Kungsgården 3 och 5]] (Kungsgården 3 och 5)


In [110]:
first.loc[["Ale","Täby","Åre"],["funktion","namn"]]

,funktion,namn
kommun,,
Ale,Repslageri (1 byggnad),"[[Repslagarmuseet]] (Starrkärr 15:4, Carlmarks..."
Täby,Slott (3 byggnader),[[Näsby slott]] (Näsbypark 57:1)
Åre,Järnvägsstation (inga registrerade byggnader),[[Enafors station]] (Handöl 6:4)


In [23]:
se_bbr.namn.str.extract('(?P<name>\[\[[\w\.\|\- ]+\]\])\,? ?(?P<name2>[ \w]+)? ?(?P<name3>\[\[[\w\.\|\- ]+\]\])? ?(?P<buildId>\([\w\.\d \|\:\-;,]+\))',expand=True)

,name,name2,name3,buildId
0,[[Ekolsunds f.d. värdshus]],NaN,NaN,(Ekolsund 3:7)
1,[[Ekolsunds slott]],NaN,NaN,(Ekolsund 1:227; f.d. 1:102)
2,[[Gryta majorsboställe]],NaN,NaN,(Gryta 5:3)
3,[[Grönsö slott]],NaN,NaN,(Grönsö 1:1)
4,[[Härnevi kvarn|Härnevi kvarn och såg]],NaN,NaN,"(Stora Härnevi 2:2, 11:1 och Kylsta 8:2)"
5,[[Kvekgården]],NaN,NaN,(Kvek 2:34 och 2:36)
6,[[Landsberga översteboställe]],NaN,NaN,(Landsberga 1:1)
7,[[Salnecke slott]],NaN,NaN,(Salnecke 1:10)
8,[[Utö hus|Utöhus]],NaN,NaN,(Utö 2:1; f.d. Utöhus 1:1)
9,[[Veckholms gamla prästgård]],NaN,NaN,(Veckholm 1:8; f.d. 1:1)


In [24]:
se_bbr.ix[5672]

bbr                                                    21300000005034
namn                       [[Långträsks kyrka]] (Piteå-Långträsk 3:1)
region-iso                                                      SE-BD
funktion                       Kyrka med begravningsplats (1 byggnad)
byggar                                                               
arkitekt                                                             
plats                                                                
kommun                                                          Piteå
lat                                                           65.3868
lon                                                           20.3236
bild                                               Långträskkyrka.jpg
commonscat                                                           
source              //sv.wikipedia.org/w/index.php?title=Lista_%C3...
changed                                           2016-06-08 06:52:31
monument_article    

Let's check If we have extracted the two always occuring fields 'name' and 'buildId' for all objects

In [48]:
se_bbr_namn = se_bbr.namn.str.extract('(?P<name>\[\[[\w`\.,\|\- ]+\]\])\,? ?(?P<name2>[ \w]+)? ?(?P<name3>\[\[[\w\.\|\- ]+\]\])? ?(?P<buildId>\([\w\.\d \|\:\-;,]+\))',expand=True)

In [64]:
len(se_bbr_namn[se_bbr_namn["name"] == pd.np.nan])

0

In [50]:
len(se_bbr_namn[se_bbr_namn["buildId"] != pd.np.nan])

5686

In [69]:
se_bbr[pd.isnull(se_bbr["namn"])]

,bbr,namn,region-iso,funktion,byggar,arkitekt,plats,kommun,lat,lon,bild,commonscat,source,changed,monument_article,registrant_url


In [60]:
len(se_bbr_namn.name.value_counts())

5148

In [62]:
len(se_bbr_namn.name2.value_counts())

89

In [63]:
len(se_bbr_namn.name3.value_counts())

45

How many objects did we get the other fields 'name2' and 'name3' for?

In [51]:
se_bbr_namn.name3.value_counts()

[[Sproge]]                                    2
[[Burs]]                                      1
[[Österviks kapell]]                          1
[[Officerspaviljongen Trianon]]               1
[[Laboratorium Chemicum]]                     1
[[Testebokvarn]]                              1
[[Flemingska palatset]]                       1
[[Kungliga Skogs- och Lantbruksakademien]]    1
[[Loftboden]]                                 1
[[Norns kapell]]                              1
[[Ekermanska huset]]                          1
[[Lurgatan]]                                  1
[[Västerleds församlingshem]]                 1
[[von Leesenska-Ribbingska huset]]            1
[[Kungsbropalatset]]                          1
[[Hejnum]]                                    1
[[Rosendals gård|Rosendal]]                   1
                                             ..
[[Eklundshof]]                                1
[[Nyköpings rådhus|rådhus]]                   1
[[Anders Olsgården]]                    

In [58]:
se_bbr[se_bbr.namn.str.contains("sproge", flags=re.IGNORECASE) == True]

,bbr,namn,region-iso,funktion,byggar,arkitekt,plats,kommun,lat,lon,bild,commonscat,source,changed,monument_article,registrant_url
2461,21300000014845,[[Stora Norrgårde]] i [[Sproge]] (Sproge Stora...,SE-I,Bondgård (4 byggnader),1700-talets slut,,,Gotland,57.26773,18.17013,,,//sv.wikipedia.org/w/index.php?title=Lista_%C3...,2016-06-08 06:52:07,Stora_Norrgårde,http://www.bebyggelseregistret.raa.se/bbr2/anl...
2467,21300000014852,[[Stymnes]] i [[Sproge]] (Sproge Stymnes 1:3),SE-I,Bondgård (2 byggnader),1870-talet,,,Gotland,57.25262,18.21990,,,//sv.wikipedia.org/w/index.php?title=Lista_%C3...,2016-06-08 06:52:07,Stymnes,http://www.bebyggelseregistret.raa.se/bbr2/anl...
2556,21300000002840,[[Sproge kyrka]] (Sproge Kyrkogården 1:1),SE-I,Kyrka med begravningsplats (1&nbsp;byggnad),,,,Gotland,57.25364,18.21091,Sproge-kyrka-Gotland-N1.jpg,,//sv.wikipedia.org/w/index.php?title=Lista_%C3...,2016-06-08 06:52:07,Sproge_kyrka,http://www.bebyggelseregistret.raa.se/bbr2/anl...
